<a href="https://colab.research.google.com/github/koominji/CrawlingLinkareer/blob/main/%EB%A7%81%EC%BB%A4%EB%A6%AC%EC%96%B4_%ED%81%AC%EB%A1%A4%EB%9F%AC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install selenium
!pip install openpyxl

     |████████████████████████████████| 958 kB 5.4 MB/s 
     |████████████████████████████████| 356 kB 52.0 MB/s 
     |████████████████████████████████| 138 kB 40.3 MB/s 
     |████████████████████████████████| 55 kB 2.7 MB/s 
     |████████████████████████████████| 3.6 MB 42.2 MB/s 
     |████████████████████████████████| 54 kB 2.8 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
requests 2.23.0 requires urllib3!=1.25.0,!=1.25.1,<1.26,>=1.21.1, but you have urllib3 1.26.7 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.


In [3]:
from selenium import webdriver
from selenium.webdriver.support.ui import Select
import time
from bs4 import BeautifulSoup
import pandas as pd
import openpyxl

In [ ]:
wb = openpyxl.Workbook()
sheet= wb.active
sheet.append(["index","view","title","content"])

#크롬드라이버 경로설정
driver_path = 'chromedriver.exe'
driver = webdriver.Chrome(executable_path=driver_path)
driver.implicitly_wait(3)
#오픈할 브라우저 url
driver.get("https://community.linkareer.com/board_Efog91")
driver.implicitly_wait(3)

time.sleep(0.5)

addrUrl = []
page = 1
# p = driver.page_source
# soup = BeautifulSoup(p,"html.parser") #html로 파싱
while page < 25: # 24페이지까지만 (수집기간 약 1년)
    try:
        time.sleep(1)
        driver.get(f"https://community.linkareer.com/board_Efog91?page={page}&field=title&word=&isNotice=false&isBest=false")
        # 링커리어 -> 취준생이야기방 게시판
        page += 1
        try:
            if driver.find_element_by_class_name("MuiTableCell-root jss414 MuiTableCell-body").text == '게시글이 없습니다.':
                # 혹시 다음페이지에 내용이 없어서 '게시글이 없습니다'가 나오면 다음페이지로 넘어가는것을 멈춤
                break
        except:
            pass
        
        for i in range(1, 21, 1): # 한 페이지에 20개의 게시글 존재
            addrUrl.append(driver.find_element_by_xpath(f'//*[@id="__next"]/div/div[4]/div/div[2]/div/div/div[2]/table/tbody/tr[{i}]/td/div/div[1]/div/a').get_attribute("href"))
            # 각 게시글 URL을 addURL리스트에 추가
        driver.switch_to.default_content()
    except:
        pass


title_list = []
content_list = []
view_count=[]
for url in addrUrl:
    time.sleep(2)
    driver.get(url) # 리스트에 저장된 각각의 게시글 URL을 엶

    title_list.append(driver.find_element_by_css_selector('div:nth-child(1) h1').text) #게시글 제목을 title_list에 추가

    pg = driver.page_source
    soup = BeautifulSoup(pg, "html.parser")  # html로 파싱
    articles = soup.find_all('a', {'class': 'jss451'})
    each_content_page = driver.page_source
    content_info_soup = BeautifulSoup(each_content_page, "html.parser")
    contents = content_info_soup.find_all('p', {'class': 'block'})

    result = ""
    
    for c in contents:
        sentence = c.text
        if sentence[:3] == "로그인":
            print("")  # '로그인'이라는 단어들어간 문장 버림(공통적으로 수집되는 불필요한 부분 제거)
        else:
            result += sentence
    content_list.append(result) # 수집된 본문+댓글을 content_list에 추가

    # 조회수 수집
    view = content_info_soup.find_all('p', {'class': 'MuiTypography-root MuiTypography-body1 MuiTypography-colorTextSecondary'})
    v = view[2].text
    view_count.append(v[3:])


#csv로 변환하기 위해 각 리스트를 데이터프레임에 담음
df = pd.DataFrame({"title": title_list, "content": content_list, "views": view_count})
df.to_csv('../output/linkareer_final.csv', encoding="utf-8-sig")
driver.quit()